In [3]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.1.0'

In [4]:
np.__version__

'1.19.5'

In [17]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as scistats
import statistics as stats
import seaborn as sns
from pathlib import Path

## Libraries, Helper Functions and High Level Variables

In [22]:
matplotlib.rcParams.update({'font.size': 12})

dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


## Getting the Data and Augmentation

In [24]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

tables['category'] = np.where(tables['scenario_name'].str.startswith('smart'),'smart','conservative')

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [26]:
tables.to_csv('tables.csv')
games.to_csv('games.csv')
hands.to_csv('hands.to_csv')

,table_id,scenario_name,player_types,category
0,1,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,10,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,100,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,101,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer,smart
0,102,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer,smart
...,...,...,...,...
0,95,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,96,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,97,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart
0,98,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer,smart


In [31]:
games.groupby('player_type').count()

,table_id,game_id,player_name,game_result,game_reason,blind_type,final_hand,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types,category
player_type,,,,,,,,,,,,,,
AlwaysCallPlayer,150000,150000,150000,150000,150000,150000,150000,150000,150000,150000,150000,150000,150000,150000
SmartPlayer,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000


In [30]:
games[(games['game_result']=='won') & (games['player_type']=='ConservativePlayer')]

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,final_hand,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types,category


## Data Samples

We have 3 individual tables: tables, games and hands.  They each represent different types of entities.  This will be explained per table.

### Table 

Our simulation software maps a scenario to a simulation, which in-turn runs a set of 2-6 players over 10 poker tables similar to simulation softwares concept of replications.  Each player within a scenario can have one of 3 personas: Smart Player, Conservative Player or Always Calls Player.  Smart Player runs a 100-card monte-carlo simulation everytime he has to make a betting decision, Conservative player ... and always calls player always calls no matter what happens.  Other types of player personas are implemented, but not used in this analysis.  We decided for this analysis to compare the smart player against the conservative player by having each player play 5 different scenarios against 1,2,3,4 and 5 always call type players. Than compare results between the two of them.  The Tables Dataframe below represents scenario and table level information eg simulation and replication info.

Fields:

* table_id -> replication id representing a poker table
* scenario_name -> represents the name of a simulation
* player_types -> a pipe delimited list of all player personas or types
* category -> if the table is part of the smart or conservative player analysis


In [ ]:
tables.head(5)

### Games

Each table is assigned a French-style poker deck of 52 cards, which it uses to play 100 sequential games of poker with.  It reshuffles the deck after each game.  The balances of the player are carried over between games.  The Games table contains the results of the games and the changes of each players balance as the 100 games progress.  It also records the overall outcome of the player within the game: won/lost and what the final hand would have been (straight, 2-pair, 3-pair etc).  The following information is contained in the Games Data Frame.

Fields:
* table_id -> a relational link to tables data frame representing the table or replication the row is associated with
* game_id -> an id representing one of 100 sequential games for a given table ordered chronologically.
* player_name -> name of the player usually player 1,2,3,4,5,6
* player_type -> the player personal or strategy used during the match
* game_result -> if the player won or lost the round
* game_reason -> did the player win or lose the match during the final hand or did he fold or alternatively was the only person that didn't fodl.
* blind type -> was the player the small or large blind
* beginning balance -> the players balance before joining the table.
* game_start_balance -> the player balance during the start of the poker game
* game_end_balance -> the player balance after the end of the poker game
* game_net_change -> the net change in players balance after the conclusion of the game
* scenario_name -> what scenario it belongs too
* player_types -> what player personas were involved in the game
* category -> what category smart or conservative player analysis dis this game belong to.


In [ ]:
games.head(5)

### Hands

Each Game of poker is actually a set of up to 12 betting decisions per player.  3 for pre-flob, 3-card community, 4-card community and 5 card community post-flops.  A player can: call/check, raise or fold the hand.  Each row in the hands dataframe represents one of these betting decisions and provides all the information a player had when making the bet.  It also shows you the result of the bet.  Useful for stepping through the players action during a game.

Fields:
* table_id -> table associated with the bet
* game_id -> game associated with the bet
* player_name -> player name associated with the bet
* player_type -> player persona or strategy used during the bet
* bet_number -> for this specific game_id what bet number this is (1st bet, 2nd bet etc)
* opponents -> number of opponents left
* call -> how much the player needs to call to stay in the game
* current -> how much the player has in the pot right now
* pot -> how large the pot or reward is
* allowed -> if the player was allowed to raise or if it was the last round of bets
* hand1 -> first card in the players hand (of 2)
* hand2 -> the second card in the players hand (of 2)
* community1-5 -> 1-5 community card in the post flop

Everything after these fields are created from a joing to both table and game for convenice.

In [ ]:
hands.head(5)

# High Level Statistics - All Simulations

We provide a high-level aggregation across all simulations to provide context into the simulation software as a whole.  Since the simulation software is highly configurable, these high level statistics are useful for gauging things like performance or to catch anomalies.

In [ ]:
player_final_hands = hands.query('community5 != "Z-N/A"').groupby(['table_id','game_id','player_name','hand1','hand2','community1','community2','community3','community4','community5','final_hand']).count().game_result.reset_index().groupby('final_hand').game_id.count().reset_index()
player_final_hands_type, player_final_hands_type_numbers = list(zip(*player_final_hands.values))
player_final_hands_type = ['player final _hand' + card for card in player_final_hands_type]

winning_hands = games.query('game_result == "won"').groupby(['game_id','final_hand']).count().groupby('final_hand').count().reset_index()[['final_hand','table_id']]
winning_hand_type, winning_hand_type_numbers = list(zip(*winning_hands.values))
winning_hand_type = ['won by ' + card for card in winning_hand_type]

simulation_overview = pd.DataFrame(
    data={
            "Metric":[
                "scenarios",
                "player strategies",
                "active players",
                "player - bet decisions",
                "tables",
                "games", 
                "average chip gain/loss",
                "average pot size",
                "unique 2-card combos",
                "unique end combos",
                "unique 3-card flops",
                "unique 4-card flops",
                "unique 5-card flops"
            ] + list(player_final_hands_type) +
                list(winning_hand_type),
            "Total":[
                tables.scenario_name.nunique(),
                games.player_type.nunique(),
                games.groupby(['table_id','player_name']).game_result.count().reset_index().shape[0],
                hands.shape[0],
                tables.table_id.nunique(),
                games.groupby(['table_id', 'game_id']).ngroups,
                round(games.game_net_change.mean(),0),
                round(hands.pot.mean(),0),
                hands.groupby(['hand1','hand2']).count().reset_index().shape[0],
                hands.query('community5 != "Z-N/A"').groupby(['hand1','hand2','community1','community2','community3','community4','community5']).count().reset_index().shape[0],
                hands.query('community1 != "Z-N/A"').groupby(['community1','community2','community3']).count().reset_index().shape[0],
                hands.query('community4 != "Z-N/A"').groupby(['community1','community2','community3','community4']).count().reset_index().shape[0],
                hands.query('community5 != "Z-N/A"').groupby(['community1','community2','community3','community4','community5']).count().reset_index().shape[0]
            ] + list(player_final_hands_type_numbers) +
                list(winning_hand_type_numbers)
    },
    columns = ['Metric','Total']
)

simulation_overview

# Performance over all Games by Player Type

Below we provide a reference for all replications (tables) within each scenario by averaging the mean net change in each persons chips over all games as well as variance and standard deviation.  We also report the players, lowest and highest balances.

In [ ]:
table_level_statistics = games.groupby(['category','scenario_name','table_id','player_name','player_type']).agg({
    "game_net_change":[np.mean,lambda col: np.var(col,ddof=1),'count'],
    "game_end_balance":[np.min,np.max]
}).reset_index().sort_values(['table_id','player_name'])

table_level_statistics.columns = ['category','scenario_name','table_id','player_name','player_type','table_mean','table_variance','count','player_lowest_balance','player_highest_balance']
table_level_statistics['confidence_interval'] = table_level_statistics.apply(lambda df: scistats.t.interval(alpha=.99,df=df['count'],loc=df['table_mean'],scale=np.sqrt(df['table_variance'])/np.sqrt(df['count'])),axis=1)
table_level_statistics['lower_bound_95%'], table_level_statistics['upper_bound_95%'] = zip(*table_level_statistics.confidence_interval)
table_level_statistics.pop('confidence_interval')
table_level_statistics



Below graph, we look at all 10 of our scenarios by player.  The top 10 graphs represent our conservative scenarios in descending order of number of players.  The bottom 10 scenarios show Smart Player in descending number of opponents.  You can see that Smart Player shifts right as more players are added, while his opponents tend to lose about the same amount of chips per table.  This is due to the fact that the always call opponents make the same type of decisions and lose about the same percent of the time, but the smart player gets to reap the benefits of those calls by on average winning a larger parts due to more contributors.

In [ ]:
fig, axis = plt.subplots(10,2,figsize=(20, 20), sharex = True)

table_level_statistics.hist(
        column='table_mean',
        by=['scenario_name','player_type'],
        grid = True,
        bins = list(range(-800,800,25)),
        ax = axis
)

for row in axis:
    for col in row:
        player_name = col.title.get_text().split(',')[1]
        players_involved = int(''.join(i for i in col.title.get_text().split(',')[0] if i.isdigit())) + 1
        graph_title = player_name[:-1] + ' - ' + str(players_involved)
        col.set_title(graph_title)

fig.text(0.5,0.05, "Bet Loss or Gain ($USD)", ha="center", va="center")
fig.text(0.05,0.5, "Number of Games", ha="center", va="center", rotation=90)


We can also look at the mean, variance and standard deviation of players across all games within a scenario.  Sometimes interesting patterns emerge.

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

game_level_statistics = games.groupby(['category','scenario_name','player_name','player_type']).agg({
    "game_net_change":[np.mean,lambda col: np.var(col,ddof=1)],
    "game_end_balance":[np.min,np.max]
}).reset_index().sort_values(['player_name'])

game_level_statistics.columns = ['category','scenario_name','player_name','player_type','game_mean','game_variance','min_balance','max_balance']

game_level_statistics.sort_values(['player_type','scenario_name'],ascending=False)

The overall game end balance is sequential, so it's a bit misleading to graph it as a box plot since each observation is not independent.  That said, the patterns in the distributions of end balances do have interesting properties in terms of ranges, mean and quartile that create an interesting observation.  Smart player's average is higher than 75% quantile of the All Call Players with the bottom of his range being about as bad as the mean of the all call players.  Smart playres end balance has significant amount of outliers sometimes 30% or more from the starting balanace.  The end balance for all call player tends to be lower, but not so drastic as the smart player.  partially due to the fact that in multiplayer games, Smart player can collect from many Always call players mitigating the damage.

In [ ]:
axis = games.boxplot(
        column=['game_end_balance'],
        by=['player_type','category'],
        figsize=(10,5),
        rot=33
)

plt.suptitle("Player Balances by Strategy and Scenario Type")
axis.set_title('Conservative and Smart Players vs Opponents')
axis.set_ylabel('Player Balance (USD) over all games')
axis.set_xlabel('Player Strategies by Scenario Type')

axis_replace={
    'AlwaysCallPlayer': 'always call player',
    'SmartPlayer': 'smart player',
    'ConservativePlayer': 'conservative player'
}

axis_cat_labels = [ax.get_text()[1:-1].split(',') for ax in axis.get_xmajorticklabels()]
axis_cat_labels = [axis_replace[ax[0]] + ' -' + ax[1] + ' scenario' for ax in axis_cat_labels]
axis.set_xticklabels(axis_cat_labels)


# Performance of Player Balances over Time

We provide the time series we used for the player balance over 100 games.  We included all 10 scenarios here.  The game_id is mapped sequentially from 1-100.  Table id is uniquely mapped as well.  On average, the player balance for smart player increases dramatically at the expense of all call players.

In [ ]:
time_series = games[['category','scenario_name','player_name','player_type','table_id','game_id','game_end_balance']]
time_series['game_id'] = (time_series['game_id'] - 1) % 100
time_series['table_id'] = (time_series['table_id'] - 1) % 10
time_series = time_series.pivot_table(index=['category','scenario_name','player_name','player_type','table_id'],columns=['game_id'],values='game_end_balance').reset_index()
time_series

Below we provide per game confidence intervals for each player in each scenario category: conservative vs smart player.  Here you can look into the data about how poker players did relatively to the number of games played.

In [ ]:
time_series = games[['category','player_type','game_id','game_end_balance']]
time_series['game_id'] = (time_series['game_id'] - 1) % 100
time_series_stats = time_series.groupby(['category','player_type','game_id']).agg(
    {'game_end_balance':[
        ['mean',np.mean],
        ['sample_var',lambda col: np.var(col,ddof=1)],
        ['N','count']]
    }
).reset_index()

time_series_stats.columns = ['category','player_type','game_id','mean','sample_var','N']
time_series_stats['confidence_interval'] = time_series_stats.apply(lambda df: scistats.norm.interval(alpha=.95, loc=df['mean'],scale=np.sqrt(df['sample_var'])/np.sqrt(df['N'])),axis=1)
time_series_stats['lower_bound_95%'], time_series_stats['upper_bound_95%'] = zip(*time_series_stats.confidence_interval)
time_series_stats.pop('confidence_interval')
time_series_stats

This graphic shows you the mean balance between all player classes within the conservative and smart player types as well as the 95th confidence interval for ending player balance over time.  As you can see, Smart Player does well at the expense of the all call players.  He seems to do so at a relatively consistent rate.

In [ ]:
fig, axis = plt.subplots(2,2,figsize=(20, 20),sharey=True)

time_series = games[['category','player_type','game_id','game_end_balance']]
time_series['game_id'] = (time_series['game_id'] - 1) % 100

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)
        
        
categories_to_plot = time_series[['category','player_type']].drop_duplicates()

plot_num = 0
for index, row in categories_to_plot.iterrows():
    ax = all_axis[plot_num]
    plot_num += 1
    player_type, category = row['player_type'], row['category']
    query_filter = 'player_type == "{}" and category == "{}"'.format(player_type,category)
    plot_data = time_series.query(query_filter)
    sns.lineplot(x=plot_data['game_id'],y=plot_data['game_end_balance'],ax=ax)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_title('Scenario: {} - Player {}'.format(category,player_type),fontsize=20)
    ax.set_ylabel('End Game Balance ($USD)',fontsize=20)
    ax.set_xlabel('Game Number (for Poker Table)',fontsize=20)



In [ ]:
fig, axis = plt.subplots(4,5,figsize=(40, 40),sharey=True)

time_series = games[['category','scenario_name','player_type','game_id','game_end_balance']]
time_series['game_id'] = (time_series['game_id'] - 1) % 100

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)
        
categories_to_plot = time_series[['category','scenario_name','player_type']].drop_duplicates()
categories_to_plot['sort_order'] =  categories_to_plot.scenario_name.str.extract('(\d+)')
categories_to_plot = categories_to_plot.sort_values(['category','player_type','sort_order'])

plot_num = 0
for index, row in categories_to_plot.iterrows():
    ax = all_axis[plot_num]
    plot_num += 1
    player_type, category, scenario_name, number = row['player_type'], row['category'], row['scenario_name'], row['sort_order']
    query_filter = 'player_type == "{}" and category == "{}" and scenario_name == "{}"'.format(player_type,category,scenario_name)
    plot_data = time_series.query(query_filter)
    sns.lineplot(x=plot_data['game_id'],y=plot_data['game_end_balance'],ax=ax)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    if plot_num > 15:
        ax.set_xlabel('Game Number',fontsize=30)
    else:
        ax.get_xaxis().set_visible(False)
    ax.set_title('{} - {}'.format(player_type,int(number) + 1),fontsize=40)
    ax.set_ylabel('End Game Balance ($USD)',fontsize=30)
    

plt.tight_layout()

# Analysis of 2 Card Games, Flob and River performance

In [ ]:
pre_flop_hands = hands.query('community1 == "Z-N/A" and bet_number == "1"')
analysis_data = pre_flop_hands[['category','player_type','opponents','hand1','hand2','game_reason','game_net_change']]
analysis_data[['rank1', 'suit1']] = analysis_data['hand1'].str.split('-', 1, expand=True)
analysis_data[['rank2', 'suit2']] = analysis_data['hand2'].str.split('-', 1, expand=True)
analysis_data['same_suit'] = analysis_data['suit1'] == analysis_data['suit2']
analysis_data

In [ ]:
analysis_data.groupby(['hand1','hand2']).game_reason.count().hist()

In [ ]:
per_game_type = analysis_data.groupby(['rank1','rank2','game_reason']).category.count().reset_index()
games_total = analysis_data.groupby(['rank1','rank2']).game_net_change.count().reset_index()
pre_flop_data = pd.merge(per_game_type,games_total).sort_values(['rank1','rank2'])
game_stats = pre_flop_data.pivot(index =['rank1','rank2','game_net_change'], columns = ['game_reason'], values =['category']).reset_index()
game_stats.columns = [c[1] if c[1] else c[0] for c in game_stats.columns]
game_stats = game_stats.fillna(0)
game_stats['fold_percent'] = game_stats['fold'] / game_stats['game_net_change']
game_stats['last_man_standing_percent'] = game_stats['last_man_standing'] / game_stats['game_net_change']
game_stats['lost_game_percent'] = game_stats['lost_game'] / game_stats['game_net_change']
game_stats['won_game_percent'] = game_stats['won_game'] / game_stats['game_net_change']
game_stats

In [ ]:
fig, axis = plt.subplots(2,2,figsize=(30, 30))

percentages = game_stats[['rank1','rank2'] + [c for c in game_stats.columns if c.endswith('percent')]].sort_values(['won_game_percent','last_man_standing_percent','fold_percent','lost_game_percent'],ascending=False)

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)
        
rank_order = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

plot_num = 0
for percent in [c for c in percentages.columns if c.endswith('percent')]:
    ax = all_axis[plot_num]
    percents_pivot = percentages.pivot(index='rank2',columns='rank1',values=percent)
    percents_pivot = percents_pivot.loc[rank_order,rank_order]
    sns.heatmap(percents_pivot, annot = True, ax = ax,cmap="YlGnBu",fmt=".0%", annot_kws={'size': 25}, cbar=False)
    ax.set_title(percent.replace('_',' '),fontsize=40)
    ax.set_xlabel('low card',fontsize=25)
    ax.set_ylabel('high card',fontsize=25)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plot_num += 1
    
plt.tight_layout()

In [ ]:
per_game_type = analysis_data.groupby(['rank1','rank2','same_suit','game_reason']).category.count().reset_index()
games_total = analysis_data.groupby(['rank1','rank2','same_suit',]).game_net_change.count().reset_index()
pre_flop_data = pd.merge(per_game_type,games_total).sort_values(['rank1','rank2','same_suit'])
game_stats = pre_flop_data.pivot(index =['rank1','rank2','game_net_change','same_suit'], columns = ['game_reason'], values =['category']).reset_index()
game_stats.columns = [c[1] if c[1] else c[0] for c in game_stats.columns]
game_stats = game_stats.fillna(0)
game_stats['fold_percent'] = game_stats['fold'] / game_stats['game_net_change']
game_stats['last_man_standing_percent'] = game_stats['last_man_standing'] / game_stats['game_net_change']
game_stats['lost_game_percent'] = game_stats['lost_game'] / game_stats['game_net_change']
game_stats['won_game_percent'] = game_stats['won_game'] / game_stats['game_net_change']
game_stats

In [ ]:
fig, axis = plt.subplots(1,2,figsize=(25, 15))

percentages = game_stats[['rank1','rank2','same_suit','won_game_percent']]

all_axis = []

for row in axis:
    all_axis.append(row)
        
rank_order = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

max_val = percentages.query("rank1 != rank2").won_game_percent.max()
min_val = percentages.query("rank1 != rank2").won_game_percent.min()

plot_num = 0
for suite_type, percent in ((False,'won_game_percent'),(True,'won_game_percent')):
    ax = all_axis[plot_num]
    filtered_percentages = percentages.query('same_suit == {} and rank1 != rank2'.format(suite_type))
    rank1_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank1'].to_list())]
    rank2_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank2'].to_list())]
    percents_pivot = filtered_percentages.pivot(index='rank2',columns='rank1',values=percent)
    percents_pivot = percents_pivot.loc[rank2_order,rank1_order]
    sns.heatmap(percents_pivot, annot = True, ax = ax,cmap="YlGnBu",fmt=".0%",vmax=max_val, vmin=min_val, annot_kws={'size': 25}, cbar=False)
    ax.set_title(percent.replace('_',' ') + ' - ' + ('same suite' if suite_type else 'opposing suite'),fontsize=25)
    ax.set_xlabel('low card',fontsize=25)
    ax.set_ylabel('high card',fontsize=25)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plot_num += 1
    print(percents_pivot.mean().mean())
    
plt.tight_layout()

In [ ]:
24 - 16

In [ ]:
per_game_type = analysis_data.groupby(['player_type','rank1','rank2','same_suit','game_reason']).category.count().reset_index()
games_total = analysis_data.groupby(['player_type','rank1','rank2','same_suit',]).game_net_change.count().reset_index()
pre_flop_data = pd.merge(per_game_type,games_total).sort_values(['player_type','rank1','rank2','same_suit'])
game_stats = pre_flop_data.pivot(index =['player_type','rank1','rank2','game_net_change','same_suit'], columns = ['game_reason'], values =['category']).reset_index()
game_stats.columns = [c[1] if c[1] else c[0] for c in game_stats.columns]
game_stats = game_stats.fillna(0)
game_stats['fold_percent'] = game_stats['fold'] / game_stats['game_net_change']
game_stats['last_man_standing_percent'] = game_stats['last_man_standing'] / game_stats['game_net_change']
game_stats['lost_game_percent'] = game_stats['lost_game'] / game_stats['game_net_change']
game_stats['won_game_percent'] = game_stats['won_game'] / game_stats['game_net_change']
game_stats

In [ ]:
fig, axis = plt.subplots(3,3,figsize=(40, 30))

percentages = game_stats[['player_type','same_suit','rank1','rank2'] + [c for c in game_stats.columns if c.endswith('percent')]].sort_values(['won_game_percent','last_man_standing_percent','fold_percent','lost_game_percent'],ascending=False)

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)

rank_order = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

suite_options = [True, False,False]
rank_options = [False,False,True]
player_type_options = ['AlwaysCallPlayer','SmartPlayer','ConservativePlayer']
options = [(suit, rank, player_type, 'won_game_percent') for suit,rank in zip(suite_options,rank_options) for player_type in player_type_options]

max_val = percentages.won_game_percent.max()
min_val = percentages.won_game_percent.min()

plot_num = 0
for suite_type,rank_type, player_type, percent in options:
    ax = all_axis[plot_num]
    if rank_type:
        query = 'same_suit == {} and player_type == "{}" and rank1 == rank2'.format(suite_type,player_type)
    else:
        query = 'same_suit == {} and player_type == "{}" and rank1 != rank2'.format(suite_type,player_type)
    filtered_percentages = percentages.query(query)
    rank1_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank1'].to_list())]
    rank2_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank2'].to_list())]
    percents_pivot = filtered_percentages.pivot(index='rank2',columns='rank1',values=percent)
    percents_pivot = percents_pivot.loc[rank2_order,rank1_order]
    sns.heatmap(percents_pivot, annot = True, ax = ax,cmap="YlGnBu",fmt=".0%",vmax=max_val, vmin=min_val, annot_kws={'size': 28}, cbar=False)
    title = player_type + ' games won - ' + ('same suite - ' if suite_type else 'opposing suite - ') + ('pairs' if rank_type else 'non-pairs') 
    ax.set_title(title,fontsize=30)
    ax.set_xlabel('low card',fontsize=25)
    ax.set_ylabel('high card',fontsize=25)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plot_num += 1
    
plt.tight_layout()

In [ ]:
fig, axis = plt.subplots(3,3,figsize=(40, 30))

percentages = game_stats[['player_type','same_suit','rank1','rank2'] + [c for c in game_stats.columns if c.endswith('percent')]].sort_values(['won_game_percent','last_man_standing_percent','fold_percent','lost_game_percent'],ascending=False)

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)

rank_order = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

suite_options = [True, False,False]
rank_options = [False,False,True]
player_type_options = ['AlwaysCallPlayer','SmartPlayer','ConservativePlayer']
options = [(suit, rank, player_type, 'fold_percent') for suit,rank in zip(suite_options,rank_options) for player_type in player_type_options]

max_val = percentages.won_game_percent.max()
min_val = percentages.won_game_percent.min()

plot_num = 0
for suite_type,rank_type, player_type, percent in options:
    ax = all_axis[plot_num]
    if rank_type:
        query = 'same_suit == {} and player_type == "{}" and rank1 == rank2'.format(suite_type,player_type)
    else:
        query = 'same_suit == {} and player_type == "{}" and rank1 != rank2'.format(suite_type,player_type)
    filtered_percentages = percentages.query(query)
    rank1_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank1'].to_list())]
    rank2_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank2'].to_list())]
    percents_pivot = filtered_percentages.pivot(index='rank2',columns='rank1',values=percent)
    percents_pivot = percents_pivot.loc[rank2_order,rank1_order]
    sns.heatmap(percents_pivot, annot = True, ax = ax,cmap="YlGnBu",fmt=".0%",vmax=max_val, vmin=min_val, annot_kws={'size': 26}, cbar=False)
    title = player_type + ' games folded - ' + ('same suite - ' if suite_type else 'opposing suite - ') + ('pairs' if rank_type else 'non-pairs') 
    ax.set_title(title,fontsize=30)
    ax.set_xlabel('low card',fontsize=25)
    ax.set_ylabel('high card',fontsize=25)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plot_num += 1
    
plt.tight_layout()

In [ ]:
fig, axis = plt.subplots(3,3,figsize=(40, 30))

percentages = game_stats[['player_type','same_suit','rank1','rank2'] + [c for c in game_stats.columns if c.endswith('percent')]].sort_values(['won_game_percent','last_man_standing_percent','fold_percent','lost_game_percent'],ascending=False)

all_axis = []

for row in axis:
    for col in row:
        all_axis.append(col)

rank_order = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

suite_options = [True, False,False]
rank_options = [False,False,True]
player_type_options = ['AlwaysCallPlayer','SmartPlayer','ConservativePlayer']
options = [(suit, rank, player_type, 'lost_game_percent') for suit,rank in zip(suite_options,rank_options) for player_type in player_type_options]

max_val = percentages.won_game_percent.max()
min_val = percentages.won_game_percent.min()

plot_num = 0
for suite_type,rank_type, player_type, percent in options:
    ax = all_axis[plot_num]
    if rank_type:
        query = 'same_suit == {} and player_type == "{}" and rank1 == rank2'.format(suite_type,player_type)
    else:
        query = 'same_suit == {} and player_type == "{}" and rank1 != rank2'.format(suite_type,player_type)
    filtered_percentages = percentages.query(query)
    rank1_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank1'].to_list())]
    rank2_order = [rank for rank in rank_order if rank in set(filtered_percentages['rank2'].to_list())]
    percents_pivot = filtered_percentages.pivot(index='rank2',columns='rank1',values=percent)
    percents_pivot = percents_pivot.loc[rank2_order,rank1_order]
    sns.heatmap(percents_pivot, annot = True, ax = ax,cmap="YlGnBu",fmt=".0%",vmax=max_val, vmin=min_val, annot_kws={'size': 26}, cbar=False)
    title = player_type + ' games folded - ' + ('same suite - ' if suite_type else 'opposing suite - ') + ('pairs' if rank_type else 'non-pairs') 
    ax.set_title(title,fontsize=30)
    ax.set_xlabel('low card',fontsize=25)
    ax.set_ylabel('high card',fontsize=25)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    plot_num += 1
    
plt.tight_layout()